# 本文档提取主城区小区aoi及房价数据
- 数据准备
    - 上海主城区行政区划数据
    - 上海全城房价数据
    - 上海全城小区数据

In [1]:
# 初始化依赖包与qgis
import os
import numpy as np
import pandas as pd


import geopandas as gpd
import matplotlib.pyplot as plt

# import pyqgis
from qgis.core import *

# Supply path to qgis install location
QgsApplication.setPrefixPath('/Applications/QGIS-LTR.app/Contents/MacOS', True)

# Create a reference to the QgsApplication.  Setting the
# second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

# Write your code here to load some layers, use processing
# algorithms, etc.

# Finally, exitQgis() is called to remove the
# provider and layer registries from memory


from qgis.analysis import QgsNativeAlgorithms
import processing
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
qgs.exitQgis()


def getAttributeTable(vlayer):
    """ get attribute table of a vector layer.
    
    Args:
        vlayer : (QgsVectorLayer instance). vector layer

    returns:
         pandas DataFrame: attribute table
    """
    attribute_dict={}
    
    for field in vlayer.fields(): # 初始化各要素属性为空列表
        attribute_dict[field.name()] = []
        
    for feature in vlayer.getFeatures(): # 遍历矢量图层中个要素
        for field in vlayer.fields(): # 遍历各属性字段
            attribute_dict[field.name()].append(feature[field.name()]) # 添加属性字段值进入字典
            
            
    return pd.DataFrame(attribute_dict) #返回DataFrame

Logged warning: Duplicate provider native registered


In [3]:
input_path = '/Users/oo/Desktop/5.Learning/esri-ces-contest/Codes/input_data'
sh_main_town_filepath = r'{}/市中心-面/center_city1.shp'.format(input_path)
sh_house_price_filepath = r'{}/shanghai_house_price.shp'.format(input_path)
sh_aoi_price_filepath = r'{}/上海市小区和写字楼/上海市AOI数据.shp'.format(input_path)


sh_main_town = QgsVectorLayer(sh_main_town_filepath, "Shanghai main town", "ogr")
sh_house_price = QgsVectorLayer(sh_house_price_filepath, "Shanghai house price", "ogr")
sh_aoi_layer = QgsVectorLayer(sh_aoi_price_filepath, "Shanghai aoi", "ogr")


In [9]:
display(getAttributeTable(sh_house_price).head())

display(getAttributeTable(sh_aoi_layer).head(1))

,name,price,county,town,address,lng_wgs84,lat_wgs84
0,上海玫瑰园商贸城,5442,奉贤,奉城,(奉贤奉城)南奉公路999弄,121.617538,30.915823
1,平安家园,5523,奉贤,四团,(奉贤四团)平福路1625弄,121.739173,30.898313
2,燎原农场,6034,奉贤,海湾,"(奉贤海湾)燎原农场运输楼, 燎原农场供销楼, 燎原农场燎原设备楼, 燎原农场医院楼, 燎原农场场部楼",121.652068,30.861294
3,褚聚街4号,6090,奉贤,奉城,(奉贤奉城)褚聚街4号,121.612381,30.902529
4,泊兰花苑,6412,松江,泗泾,(松江泗泾)张泾路1339弄,121.275846,31.104524


,name,province,city,area,address,uid,detail_url,tag,wgs84_lng,wgs84_lat,行业大类,行业子类,Get_time
0,盛世年华小区,上海市,上海市,浦东新区,上海市浦东新区洋泾街道,1e0f0deb63876ada1d1d1148,"https://map.baidu.com/poi/怡家园/@13530401.154588437,3640824.8874104554,18.43z?uid=1e0f0deb63876ada1d1d1148&info_merge=1&isBizPoi=false&ugc_type=3&ugc_ver=1&device_ratio=2&compat=1&pcevaname=pc4.1&querytype=detailConInfo&da_src=shareurl",房地产;住宅区,NULL,NULL,房地产,住宅区,2021-10-25 09:56:15
